<a href="https://colab.research.google.com/github/silentfortin/ai-portfolio/blob/main/01-contactease-python/ContactEase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ContactEase – Contact Manager in Python
👨‍💻 Project developed during the AI Engineering Master – Week 1

🔗 GitHub Repo: [ai-portfolio](https://github.com/silentfortin/ai-portfolio/)


In [3]:
import uuid
import json

from json.decoder import JSONDecodeError

In [ ]:
# Class representing a User with name and surname
class User:
  # Constructor to initialize name and surname
  def __init__(self, name, surname):
    self.name = name
    self.surname = surname

class Contact:
    def __init__(self, name, surname, phone_number):
        self.id = str(uuid.uuid4())
        self.name = name
        self.surname = surname
        self.phone = phone_number

class ContactsBook:

  def __init__(self):
    self.contacts = []

  def add_contact(self, contact):
    self.contacts.append(contact)
    self.save_to_file()
    print(f'{contact.name} {contact.surname} has been added.')

  def remove_contact(self, contact_to_remove):
      for contact in self.contacts:
          if contact.id == contact_to_remove.id:
              self.contacts.remove(contact)
              # saving
              self.save_to_file()
              print(f"Contact {contact.name} {contact.surname} removed.")
              return
      print("Contact not found.")

  """
  User will insert 'mod' which is the element that wants to modify and then
  the new value 'updated_var'. When the case match, the var is modified and
  the file pudated
  """
  def modify_contact(self, contact, mod, updated_var):
      match mod:
        case 'name':
          contact.name = updated_var
        case 'surname':
          contact.surname = updated_var
        case 'phone_num':
          contact.phone = updated_var
        case _:
            print(f"Field '{field}' is not valid. Allowed: name, surname, phone.")
            return

      self.save_to_file()

  def search_contact(self, name, surname):
    for contact in self.contacts:
      if contact.name.lower() == name.lower() and contact.surname.lower() == surname.lower():
        return contact

  def show_all_contacts(self):
      for contact in self.contacts:
          print(f"{contact.name} {contact.surname} | Tel: {contact.phone}")

  def save_to_file(self, filename="contacts_book.json"):
      try:
        with open(filename, "w") as f:
          # Convert each Contact object to a dictionary
          data = [vars(c) for c in self.contacts]
            # Save the list of contacts as JSON with indentation
          json.dump(data, f, indent=2)
      except JSONDecodeError as e:
          print(f'Error: Failed to encode contacts to JSON. Details: {e}')
      except Exception as e:
          print(f'Unexpected error while saving contacts: {e}')

  def load_from_file(self, filename="contacts_book.json"):
      try:
        with open(filename, "r") as file:
          # Load list of dictionaries from JSON
          data = json.load(file)
          # Recreate Contact objects from dictionaries (that's why I'm using **c)
          self.contacts = [Contact(**c) for c in data]
      except FileNotFoundError:
          print(f'Warning: File "{filename}" not found. Starting with an empty contact list.')
          self.contacts = []
      except JSONDecodeError as e:
          print(f'Warning: File contains invalid JSON. Details: {e}')
          self.contacts = []
      except Exception as e:
          print(f'Unexpected error while loading contacts: {e}')
          self.contacts = []


In [4]:
help(json.load)

Help on function load in module json:

load(fp, *, cls=None, object_hook=None, parse_float=None, parse_int=None, parse_constant=None, object_pairs_hook=None, **kw)
    Deserialize ``fp`` (a ``.read()``-supporting file-like object containing
    a JSON document) to a Python object.
    
    ``object_hook`` is an optional function that will be called with the
    result of any object literal decode (a ``dict``). The return value of
    ``object_hook`` will be used instead of the ``dict``. This feature
    can be used to implement custom decoders (e.g. JSON-RPC class hinting).
    
    ``object_pairs_hook`` is an optional function that will be called with the
    result of any object literal decoded with an ordered list of pairs.  The
    return value of ``object_pairs_hook`` will be used instead of the ``dict``.
    This feature can be used to implement custom decoders.  If ``object_hook``
    is also defined, the ``object_pairs_hook`` takes priority.
    
    To use a custom ``JSONDecod